# Get_Original_AlphaNet_Data

In [1]:
Alpha_Name = "AlphaNet_Original_Input_12.22_1d_return"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "1d_ret"

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp

# Get_Original_AlphaNet_Data

In [8]:
dir_list = ['Data','Cov','Corr','Return','Std','Zscore','Decay']
for i in dir_list:
    os.makedirs(path+i)

In [12]:
df = pd.read_csv(path+"data.csv")
df.dropna(inplace=True,axis=0,how="any")
df

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
0,2015-01-06 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,-0.018297
1,2015-01-06 09:30:00,2,1774.23,1885.20,1753.27,1838.34,1823.031224,6560835.70,7.2662,6.7834,8.1741,-0.023288
2,2015-01-06 09:30:00,4,63.31,64.62,62.18,63.76,63.277989,28095.12,0.7707,3.3496,4.5343,0.051118
3,2015-01-06 09:30:00,5,38.00,38.00,38.00,38.00,37.997172,0.00,0.0000,0.0000,0.0000,0.000000
4,2015-01-06 09:30:00,6,225.64,234.85,223.10,225.00,228.438278,1078647.12,0.4255,8.0562,13.2751,-0.028571
...,...,...,...,...,...,...,...,...,...,...,...,...
5357756,2021-07-01 09:30:00,688700,47.80,54.50,47.03,53.38,50.715300,64336.83,14.5494,19.2222,19.2222,-0.040395
5357757,2021-07-01 09:30:00,688777,90.21,95.08,89.32,94.42,93.250464,19001.61,4.6111,4.6288,4.6288,-0.005927
5357758,2021-07-01 09:30:00,688788,138.07,149.35,137.11,146.22,145.197985,13572.56,5.1954,7.7977,7.7977,-0.025238
5357759,2021-07-01 09:30:00,688819,44.04,45.64,43.95,45.15,45.026221,47249.01,1.8544,4.6754,4.6754,-0.022042


In [13]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(ticker):
    one_data = df[df['ticker'] == ticker]
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
    return concat_df

In [14]:

if __name__=="__main__":
    pool = mp.Pool(10)
    result = {}
    t1 = time.time()
    for ticker in tqdm(df['ticker'].drop_duplicates()):
        result[ticker] = (pool.apply_async(test, args=(ticker,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)
final_df

100%|██████████| 4427/4427 [00:00<00:00, 358265.49it/s]

并行执行时间： 50


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,open,...,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,1126.20,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.000717
2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,4.3668,1105.60,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.005731
2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,3.4269,1101.33,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.017094
2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,2.8375,1058.70,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.018841
2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,5.0564,1056.57,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 09:30:00,605339,20.38,24.45,20.38,24.45,24.423600,4472.57,43.9929,0.7040,0.7040,26.90,...,44.75,45.44,44.10,44.99,44.862500,43863.72,0.6938,6.9045,6.9045,-0.034144
2021-07-01 09:30:00,605339,26.90,26.90,26.90,26.90,26.900000,1373.52,10.0204,0.2162,0.2162,29.59,...,44.81,44.88,42.70,43.18,43.469700,72397.12,-4.0231,11.3958,11.3958,0.000231
2021-06-30 09:30:00,688314,100.20,105.00,86.78,92.05,92.899300,106963.24,430.8535,90.5821,90.5821,94.00,...,115.11,115.78,110.21,112.00,112.765600,6286.46,-4.1014,5.3237,5.3237,-0.055645


In [15]:
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df.dropna(axis=0,inplace=True) # delete none
final_df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,261,262,263,264,265,266,267,268,269,target
0,2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.000717
1,2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.005731
2,2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.017094
3,2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.018841
4,2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230281,2021-06-30 09:30:00,605339,20.38,24.45,20.38,24.45,24.423600,4472.57,43.9929,0.7040,...,44.75,45.44,44.10,44.99,44.862500,43863.72,0.6938,6.9045,6.9045,-0.034144
5230282,2021-07-01 09:30:00,605339,26.90,26.90,26.90,26.90,26.900000,1373.52,10.0204,0.2162,...,44.81,44.88,42.70,43.18,43.469700,72397.12,-4.0231,11.3958,11.3958,0.000231
5230283,2021-06-30 09:30:00,688314,100.20,105.00,86.78,92.05,92.899300,106963.24,430.8535,90.5821,...,115.11,115.78,110.21,112.00,112.765600,6286.46,-4.1014,5.3237,5.3237,-0.055645
5230284,2021-07-01 09:30:00,688314,94.00,106.12,83.00,102.50,92.323600,80295.81,11.3525,67.9987,...,112.32,112.32,103.50,105.64,107.011300,9925.98,-5.6786,8.4058,8.4058,0.008768


In [20]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20150101,20150301,20150601,20150901,
             20160101,20160301,20160601,20160901,
             20170101,20170301,20170601,20170901,
             20180101,20180301,20180601,20180901,
             20190101,20190301,20190601,20190901,
             20200101,20200301,20200601,20200901,
             20210101,20210301,20210601]

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].to_parquet(
        path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print("Save to ",path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par")
    return None

pool = mp.Pool(10)
t1 = time.time()
for i in tqdm(range(len(time_list)-1)):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    pool.apply_async(save, args=(start_time,end_time))  #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 3323.94it/s]


Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-01-01_2015-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-01-01_2016-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2017-01-01_2017-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-03-01_2015-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-06-01_2015-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2017-03-01_2017-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-09-01_2016-01-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-06-01_2016-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-03-01_2016-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-09-01_2017-01-01.par


# Get Factor

In [21]:
day = 30
stride = 10
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(data_length, stride):
    if data_length % stride == 0:
        step_list = list(range(0, data_length + stride, stride))
    elif data_length % stride <= 5:
        mod = data_length % stride
        step_list = list(range(0, data_length - stride, stride)) + [data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
    return step_list

In [22]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 147766.81it/s]


['2015-01-01_2015-03-01.par',
 '2016-01-01_2016-03-01.par',
 '2017-01-01_2017-03-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-03-01_2015-06-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2017-03-01_2017-06-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-06-01_2017-09-01.par',
 '2018-01-01_2018-03-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-06-01_2020-09-01.par',
 '2021-01-01_2021-03-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-03-01_2021-06-01.par']

In [23]:
# Read Data

def cov(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis=4, keepdims=True)
        spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                keepdims=True)  # (2000, 1, 36, 1, 1)
        l.append(cov)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def corr(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov") + name)
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    return l

def std(x,num,num_rev,step_list):
    """Main Function"""
    std = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        std1 = sub_data1.std(axis=3, keepdims=True)
        std.append(std1)
    return np.squeeze(np.array(std)).transpose(1, 2, 0)

def zscore(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        mean = sub_data1.mean(axis=3, keepdims=True)
        std = sub_data1.std(axis=3, keepdims=True)
        z_score = np.divide(mean, std, out=np.zeros_like(mean, dtype=np.float64), where=std!=0)
        l.append(z_score)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def ret(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        ret = np.divide(sub_data1[:, :, :, -1], sub_data1[:, :, :, 0], out=sub_data1[:, :, :, -1], where=sub_data1[:, :, :, 0]!=0) - 1
        l.append(ret)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def decay(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        time_spread = end - start
        weight = np.arange(1, time_spread + 1)
        weight = weight / (weight.sum())
        sub_data1 = (x[:, :, :, start:end] * weight).sum(axis=3, keepdims=True)
        l.append(sub_data1)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def test(input_path,name,output_path,method = None):
#     print("read" + input_path + name)
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(data_length, stride)

    """Main Function"""
    if method == "Cov":
        l = cov(x,num,num_rev,step_list)
    if method == "Corr":
        l = cov(x,num,num_rev,step_list)
    if method == "Std":
        l = std(x,num,num_rev,step_list)
    if method == "Zscore":
        l = zscore(x,num,num_rev,step_list)
    if method == "Return":
        l = ret(x,num,num_rev,step_list)
    if method == "Decay":
        l = decay(x,num,num_rev,step_list)

    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [24]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Std','Zscore','Return','Decay']
for i in dir_list:
    for alpha in path_list:
        output = path + i + "/" 
        pool.apply_async(test, args=(data_path,alpha,output,i))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

  0%|          | 0/3 [00:00<?, ?it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Cov/2015-01-01_2015-03-01.par


 67%|██████▋   | 2/3 [00:13<00:06,  6.59s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Cov/2016-01-01_2016-03-01.par


 67%|██████▋   | 2/3 [00:13<00:06,  6.70s/it]

...

 /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Decay/2019-09-01_2020-01-01.par


 67%|██████▋   | 2/3 [00:00<00:00,  4.85it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Decay/2021-03-01_2021-06-01.par


100%|██████████| 3/3 [00:00<00:00,  4.84it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Decay/2020-09-01_2021-01-01.par
并行执行时间： 105


In [25]:
t1 = time.time()
pool = mp.Pool(10)
for alpha in path_list:
    output = path + "Corr" + "/" 
    pool.apply_async(test, args=(data_path,alpha,output,"Corr"))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 3/3 [00:01<00:00,  2.72it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2015-01-01_2015-03-01.par


100%|██████████| 3/3 [00:08<00:00,  3.00s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2017-01-01_2017-03-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2016-01-01_2016-03-01.par


 67%|██████▋   | 2/3 [00:07<00:03,  3.79s/it]

...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2021-01-01_2021-03-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2019-09-01_2020-01-01.par


 67%|██████▋   | 2/3 [00:17<00:08,  8.99s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2020-06-01_2020-09-01.par


100%|██████████| 3/3 [00:26<00:00,  8.80s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2020-09-01_2021-01-01.par
并行执行时间： 92


# Feat Concat ( Save to Feat_Concat file)

In [26]:
os.mkdir(path+"Feat_Concat")

In [3]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 218103.81it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [5]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Corr','Std','Zscore','Return','Decay']

final = []
def test(alpha,dir_list):
    feat_cat = []
    for i in dir_list:
        feat_cat.append(pd.read_parquet(path + i + "/" + alpha).set_index(['timestamp','ticker']))
    feat_cat = pd.concat(feat_cat,axis=1)
    feat_cat.columns = [str(i) for i in range(feat_cat.shape[1])]
    feat_cat.reset_index().to_parquet(path+"Feat_Concat/"+alpha)
    print(path+"Feat_Concat/"+alpha)
    return None

for alpha in tqdm(path_list):
#     test(alpha,dir_list)
    pool.apply_async(test, args=(alpha,dir_list))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 34818.62it/s]


/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-01-01_2015-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2016-01-01_2016-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2017-01-01_2017-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-03-01_2015-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2018-01-01_2018-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2017-06-01_2017-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2016-03-01_2016-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2016-06-01_2016-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2017-03-01_2017-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-06-01_2015-09-01.par
/home/wuwenjun/Data/

# Feat Daily Norm ( Save to Feat_Concat_Norm_Daily)

In [ ]:
# import time
# import os
# from Research.backtest.bt import BTDaily
# from Research.feature.ft import FeatureAnalysis
# import Research.utils.namespace as namespace
# import Research.utils.normalization as norm
# from Platform.database.mysql import MysqlAPI
# from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
# from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
# import DataAPI
# from os import walk
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from progressbar import ProgressBar
# from tqdm import tqdm
# Alpha_Name = "AlphaNet_Original_Input_12.14"
# path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
# target = "5d_ret"

In [ ]:
# os.mkdir(path+"Feat_Concat_Norm_Daily")

In [ ]:
# path_list = []
# data_path = path + "Data/"
# dataframe_list = pd.DataFrame()
# for f, _, i in walk(data_path):
#     for j in tqdm(i):
#         path_list.append(j)
# path_list

In [ ]:
# import multiprocessing as mp


# """ Multi Processing"""
# pool = mp.Pool(10)
# t1 = time.time()
# def test(file):
#     df = pd.read_parquet(path + "Feat_Concat/" + file)
#     final = []
#     for day in tqdm(df['timestamp'].drop_duplicates()):
#         array = df[df['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = array.index
        
#         # Batch
#         array = torch.from_numpy(np.array(array)).to(torch.float) 
#         batch = nn.BatchNorm1d(array.shape[1])
#         array = batch(array)
#         array = pd.DataFrame(array.detach().numpy())
#         array.index = index
#         final.append(array)
        
#     final = pd.concat(final,axis=0)
#     final.columns = [str(i) for i in range(final.shape[1])]
#     final.reset_index(inplace=True)
#     final.to_parquet(path + "Feat_Concat_Norm_Daily/" + file)
#     print("Save to : ", path + "Feat_Concat_Norm_Daily/" + file)
#     return None

# for file in path_list:
# #     test(file)
#     pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
# pool.close()
# pool.join()

# t2 = time.time()
# print("并行执行时间：", int(t2 - t1))

# Pooling ( Save to Pooling)

In [5]:
os.mkdir(path+"Pooling")

In [8]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 146378.39it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [10]:
""" Multi Processing"""
pool = mp.Pool(10)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))
    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.columns = [str(i) for i in range(array.shape[1])]
    array.reset_index(inplace=True)
#     # Daily Norm
#     final = []
#     for day in tqdm(array['timestamp'].drop_duplicates()):
#         df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = df.index
#         # Batch
#         df = torch.from_numpy(np.array(df)).to(torch.float) 
#         batch = nn.BatchNorm1d(df.shape[1])
#         df = batch(df)
#         df = pd.DataFrame(df.detach().numpy())
#         df.index = index
#         final.append(df)

#     final = pd.concat(final,axis=0)
#     final.reset_index(inplace=True)
    array.to_parquet(path + "Pooling/" + file)
    print("Save to : ", path + "Pooling/" + file)
    return None
for file in path_list:
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2016-06-01_2016-09-01.par
Save to :  Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2015-06-01_2015-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling/2015-09-01_2016-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_

# Normalize Input and Concat

In [19]:
os.mkdir(path+"Final")

FileExistsError: [Errno 17] File exists: '/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final'

In [13]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 177608.96it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [17]:
def test(file):
    array = pd.concat([pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"]),
                   pd.read_parquet(path + "Pooling/" + file).set_index(["timestamp","ticker"])],axis=1)
    array.columns = [str(i) for i in range(array.shape[1])]

    array.reset_index().to_parquet(path +"Final/" + file)
#     array.reset_index(inplace=True)
#     # Stock Norm
#     final = []
#     for day in tqdm(array['ticker'].drop_duplicates()):
#         df = array[array['ticker'] == day].set_index(['timestamp','ticker'])
#         index = df.index
#         # Batch
#         df = torch.from_numpy(np.array(df)).to(torch.float) 
#         batch = nn.BatchNorm1d(df.shape[1])
#         df = batch(df)
#         df = pd.DataFrame(df.detach().numpy())
#         df.index = index
#         final.append(df)
#     final = pd.concat(final,axis=0)
#     array = final.reset_index()
#     # Daily Norm
#     final = []
#     for day in tqdm(array['timestamp'].drop_duplicates()):
#         df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = df.index
#         # Batch
#         df = torch.from_numpy(np.array(df)).to(torch.float) 
#         batch = nn.BatchNorm1d(df.shape[1])
#         df = batch(df)
#         df = pd.DataFrame(df.detach().numpy())
#         df.index = index
#         final.append(df)
#     final = pd.concat(final,axis=0)
#     final.reset_index().to_parquet(path +"Final/" + file)
    print("Save to : ",path +"Final/" + file)
    return None

In [18]:
pool = mp.Pool(10)
t1 = time.time()
for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 3042.91it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2018-01-01_2018-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Fi

In [19]:
pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2020-09-01_2021-01-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,...,638,639,640,641,642,643,644,645,646,647
0,2020-09-01 09:30:00,1,2958.241140,1520.949191,1337.703211,2697.080646,1287.593226,1299.543283,2613.349626,1403.775333,...,1.309742e+06,1.507420,0.563644,-0.246320,0.912427,0.770484,0.674922,2.058518,1.738312,1.522747
1,2020-09-02 09:30:00,1,1896.292798,2768.892622,1212.431044,1547.343153,2470.737202,1163.252267,1508.221538,2712.761349,...,1.229030e+06,1.252091,0.754327,0.498911,0.877560,0.775732,0.633335,1.979875,1.750156,1.428915
2,2020-09-03 09:30:00,1,951.749956,3396.062976,1229.971872,627.068078,3257.099113,1122.038558,630.251317,3301.169776,...,1.315482e+06,1.049627,0.655068,0.189658,0.859575,0.780193,0.677884,1.939313,1.760221,1.529425
3,2020-09-04 09:30:00,1,546.429797,3325.811419,1437.283789,332.789021,3147.682306,1361.166391,343.827242,3031.520622,...,1.324591e+06,0.454209,0.304883,0.197627,0.833607,0.765572,0.682575,1.880740,1.727238,1.540018
4,2020-09-07 09:30:00,1,458.803053,2539.896856,1412.689396,384.127393,2564.194394,1341.665664,421.012334,2271.803289,...,1.267417e+06,0.229444,0.142725,-0.004476,0.798525,0.727273,0.653116,1.801582,1.640827,1.473551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326674,2020-12-31 09:30:00,300907,24.314772,7.891882,6.466808,17.624994,8.116586,6.049856,21.087967,7.678293,...,2.924527e+04,-0.894798,-1.594832,-2.173645,29.358042,18.910109,12.848311,29.358042,18.910109,12.848311
326675,2020-12-30 09:30:00,300909,40.160736,13.990733,17.356172,29.914047,13.631450,16.544050,31.626932,13.840400,...,2.669477e+04,2.097278,-0.527316,-2.244793,23.860113,16.295896,11.157609,23.860113,16.295896,11.157609
326676,2020-12-31 09:30:00,300909,29.224981,13.299311,19.566280,22.561846,13.818411,18.907669,20.874516,13.171600,...,2.588386e+04,-0.828080,-1.445518,-1.785225,22.407971,15.474451,10.818673,22.407971,15.474451,10.818673
326677,2020-12-30 09:30:00,688219,22.375567,3.302263,1.686413,11.657156,3.012052,1.526554,14.783222,2.791529,...,2.983635e+04,4.295062,0.104099,-2.194998,25.080547,13.951124,7.191045,25.080547,13.951124,7.191045


# Normalize Target and Concat

In [20]:
dataframe_list = pd.read_csv(path + "data.csv").set_index(["timestamp","ticker"])
df = dataframe_list[target].unstack()
df = (df.T - df.mean(axis=1)) / df.std(axis=1)
df = df.T.stack()
df = pd.DataFrame(df,columns=["target"])
# to date time
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index(["timestamp","ticker"],inplace=True)
df

target
timestamp           ticker          
2015-01-06 09:30:00 1      -1.212328
                    2      -1.374955
                    4       1.049421
                    5      -0.616170
                    6      -1.547116
...                              ...
2021-07-01 09:30:00 688700 -0.708172
                    688777  0.318826
                    688788 -0.256562
                    688819 -0.161340
                    688981 -0.073268

[5357704 rows x 1 columns]

In [26]:
def test(i):
    dataframe_list = pd.read_parquet(path +"Final/" + i).set_index(["timestamp","ticker"])
    dataframe_list = pd.merge(dataframe_list,df,how="left",left_index=True,right_index=True)
    dataframe_list.reset_index().to_parquet(path +"Final/" + i)
    print("Save to : ",path +"Final/" + i)
    return None

In [27]:
pool = mp.Pool(10)
t1 = time.time()
for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 48597.11it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2018-01-01_2018-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Fi

In [28]:
pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Final/2021-03-01_2021-06-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,...,639,640,641,642,643,644,645,646,647,target
0,2021-03-01 09:30:00,1,9144.412658,9609.107256,17343.679924,9540.948282,9102.443989,15908.507237,8514.660236,9231.267356,...,1.307375,0.120016,-1.644002,0.760896,0.732935,0.694002,1.716718,1.653629,1.565764,-0.588642
1,2021-03-02 09:30:00,1,6637.811554,7912.787007,18722.978546,7451.160581,8497.723798,15946.666630,6396.532033,9159.527843,...,0.975315,0.180155,-1.311880,0.731909,0.702050,0.656478,1.651325,1.583956,1.481113,0.294105
2,2021-03-03 09:30:00,1,5555.828116,6921.384569,16663.091471,6490.130298,8700.591811,13165.856151,5602.380020,7657.697371,...,0.624229,0.110108,-0.885193,0.735155,0.682751,0.635313,1.658645,1.540413,1.433373,1.756611
3,2021-03-04 09:30:00,1,4687.043733,5295.081338,11296.261012,5885.388478,6678.399380,10494.377014,5005.630080,5319.119058,...,1.044655,0.468128,-0.121302,0.775222,0.712540,0.631351,1.749042,1.607624,1.424427,0.234582
4,2021-03-05 09:30:00,1,2037.705867,2930.039160,7954.515162,3615.762398,5108.580811,7482.575589,2055.044289,3147.225293,...,0.624575,0.421260,0.175636,0.749136,0.691010,0.584184,1.690187,1.559035,1.317993,-1.036358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259327,2021-05-31 09:30:00,300967,7.380674,0.324353,0.050078,5.105200,0.248538,0.019561,5.002978,0.139076,...,-0.397500,-0.987333,-2.156811,26.311845,19.482492,11.049140,26.311845,19.482492,11.049140,-0.109409
259328,2021-05-28 09:30:00,688315,2.111999,8.528012,15.446156,2.396701,11.319738,16.254833,1.291436,6.639382,...,4.010451,2.106333,0.177544,26.446916,16.950781,8.918751,26.446916,16.950781,8.918751,-1.004314
259329,2021-05-31 09:30:00,688315,3.935878,2.452102,9.494124,3.512362,4.373798,11.571151,3.763251,2.673604,...,5.349111,2.252524,0.594142,24.891625,15.775256,8.960518,24.891625,15.775256,8.960518,5.393112
259330,2021-05-28 09:30:00,688663,1.414149,0.559479,0.557673,0.677036,0.435687,0.308656,0.581862,0.423508,...,0.467975,0.167139,-0.066093,22.598884,15.925854,11.658533,22.598884,15.925854,11.658533,-1.748636
